In [1]:
# libraries used
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.svm import LinearSVC, SVC
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier, StackingClassifier

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.neighbors import KNeighborsClassifier


from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
import matplotlib.patches as patches
from aif360.algorithms.preprocessing import Reweighing
#from packages import *
#from ml_fairness import *
import matplotlib.pyplot as plt
import seaborn as sns



from IPython.display import Markdown, display

import ipynbname
nb_fname = ipynbname.name()
nb_path = ipynbname.path()

import pickle

In [2]:
TM1 = pd.read_csv('Titanic/Results/Stacking/1-titanic-eda-and-feature-engineering.csv')
TM2 = pd.read_csv('Titanic/Results/Stacking/2-top-1-titanic-solution.csv')
TM3 = pd.read_csv('Titanic/Results/Stacking/3-introduction-to-ensembling-techniques.csv')
TM4 = pd.read_csv('Titanic/Results/Stacking/4-stacked-classifier-top-10-on-lb.csv')
TM5 = pd.read_csv('Titanic/Results/Stacking/5-ultimate-titanic-guide-for-beginners.csv')
TM6 = pd.read_csv('Titanic/Results/Stacking/6-titanic-complete-guide-to-top-3.csv')
#AM0 = pd.read_csv('AdultNoteBook/Results/Stacking/0-income-classification-using-meta-learning.csv')

In [3]:
TM1_model = pd.read_pickle('Titanic/Results/Stacking/1-titanic-eda-and-feature-engineering.pkl')
TM2_model = pd.read_pickle('Titanic/Results/Stacking/2-top-1-titanic-solution.pkl')
TM3_model = pd.read_pickle('Titanic/Results/Stacking/3-introduction-to-ensembling-techniques.pkl')
TM4_model = pd.read_pickle('Titanic/Results/Stacking/4-stacked-classifier-top-10-on-lb.pkl')
TM5_model = pd.read_pickle('Titanic/Results/Stacking/5-ultimate-titanic-guide-for-beginners.pkl')
TM6_model = pd.read_pickle('Titanic/Results/Stacking/6-titanic-complete-guide-to-top-3.pkl')

In [26]:
TM6_data = pd.read_pickle('Titanic/Results/Stacking/6-titanic-complete-guide-to-top-3_Train.pkl')
df = TM6_data.convert_to_dataframe()
df[0].columns

Index(['Fare', 'Age_cat', 'Ticket_frequency', 'Ticket_extracted',
       'Name_length', 'Family_Survival', 'Pclass_2', 'Pclass_3', 'Sex_male',
       'Embarked_Q', 'Embarked_S', 'Deck_DE', 'Deck_FG', 'Deck_M',
       'Title_Miss.', 'Title_Mr.', 'Title_Mrs.', 'Title_rare',
       'Family_size_cat_large_family', 'Family_size_cat_small_family',
       'Survived'],
      dtype='object')

In [31]:
TM4_data =  pd.read_pickle('Titanic/Results/Stacking/4-stacked-classifier-top-10-on-lb_Train.pkl')
df1 = TM4_data.convert_to_dataframe()
df1[0].columns

Index(['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'NlengthD', 'FsizeD',
       'Title', 'Deck', 'NameLength', 'TicketNumber', 'Survived'],
      dtype='object')

In [4]:
final = [TM1['classifier'], TM2['classifier'], TM3['classifier'], TM4['classifier'], TM5['classifier'], TM6['classifier']]

In [5]:
model_estimators = [TM1_model.estimators, TM2_model.estimators, TM3_model.estimators, TM4_model.estimators, TM5_model.estimators, TM6_model.estimators]

In [6]:
df = pd.DataFrame({'TM1':TM1['classifier'], 'TM2':TM2['classifier'],'TM3':TM3['classifier'],'TM4':TM4['classifier'],'TM5':TM5['classifier'],'TM6':TM6['classifier'] })
a_series = pd.Series(model_estimators, index = df.columns)
df = df.append(a_series, ignore_index=True)

In [7]:
df.index = ['accuracy', 'f1', 'spd', 'eod', 'aod', 'di', 'ti', 'estimators']

In [8]:
df

,TM1,TM2,TM3,TM4,TM5,TM6
accuracy,0.802239,0.847015,0.835821,0.828358,0.735849,0.876866
f1,0.741463,0.8,0.752809,0.762887,0.631579,0.835821
spd,-0.716873,-0.843457,-0.773227,-0.780652,-0.445653,-0.755544
eod,-0.638537,-0.72,-0.616667,-0.622857,-0.32605,-0.645161
aod,0.554231,0.675373,0.609903,0.625369,0.304356,0.504019
di,-2.40658,-2.64157,-2.3976,-2.46802,-1.66571,-2.31456
ti,0.177794,0.113712,0.116935,0.112602,0.248921,0.0955157
estimators,"[(logreg, LogisticRegression(C=0.28, l1_ratio=...","[(SVM, SVC(C=5, degree=2, gamma=0.1, kernel='p...","[(xgb, XGBClassifier(base_score=None, booster=...","[(svc, SVC(C=50, degree=1, gamma='auto', proba...","[(rf_1, RandomForestClassifier(n_estimators=40...","[(logreg, LogisticRegression(C=0.28, l1_ratio=..."


In [10]:
df.to_csv("Stacking_Titanic_Anlaysis.csv")